## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Korla,CN,41.7597,86.1469,77.56,20,100,5.41,overcast clouds
1,1,Ribeira Grande,PT,38.5167,-28.7000,66.90,67,94,5.59,overcast clouds
2,2,Saint Anthony,US,45.0205,-93.2180,91.74,20,1,4.00,clear sky
3,3,Cape Town,ZA,-33.9258,18.4232,57.11,98,18,3.44,fog
4,4,Haines Junction,CA,60.7522,-137.5108,67.95,34,100,2.30,overcast clouds
5,5,Chuy,UY,-33.6971,-53.4616,50.32,85,59,7.45,broken clouds
6,6,Rongcheng,CN,30.6340,117.8517,69.64,98,100,2.28,heavy intensity rain
7,7,Novaya Zaimka,RU,56.4857,66.9241,44.58,67,51,7.43,broken clouds
8,8,Puerto Ayora,EC,-0.7393,-90.3518,76.96,98,53,7.00,light rain
9,9,Nyimba,ZM,-14.5566,30.8149,67.50,47,100,6.08,overcast clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input('What is the minimum temperature you want in your trip?'))
max_temp = int(input('What is the maximum temperature you want in your trip?'))

What is the minimum temperature you want in your trip?75
What is the maximum temperature you want in your trip?90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                 (city_data_df['Max Temp'] >= min_temp)]
preferred_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Korla,CN,41.7597,86.1469,77.56,20,100,5.41,overcast clouds
8,8,Puerto Ayora,EC,-0.7393,-90.3518,76.96,98,53,7.00,light rain
10,10,Havelock,US,34.8791,-76.9013,88.05,57,7,5.01,clear sky
17,17,Jamestown,US,42.0970,-79.2353,78.12,68,90,13.80,overcast clouds
18,18,Padang,ID,-0.9492,100.3543,76.66,69,95,3.83,overcast clouds
21,21,Kapaa,US,22.0752,-159.3190,82.72,73,20,3.00,few clouds
30,30,Malartic,CA,48.1335,-78.1328,80.42,71,54,13.27,broken clouds
38,38,Atuona,PF,-9.8000,-139.0333,79.30,77,17,21.41,few clouds
41,41,Labuhan,ID,-6.8844,112.2051,77.49,87,100,4.59,light rain
42,42,Albany,US,42.6001,-73.9662,80.55,56,100,6.87,overcast clouds


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities = preferred_cities.dropna() #there weren't any null values so nothing changed
preferred_cities.count()

City_ID        225
City           225
Country        225
Lat            225
Lng            225
Max Temp       225
Humidity       225
Cloudiness     225
Wind Speed     225
Description    225
dtype: int64

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Korla,CN,77.56,overcast clouds,41.7597,86.1469,
8,Puerto Ayora,EC,76.96,light rain,-0.7393,-90.3518,
10,Havelock,US,88.05,clear sky,34.8791,-76.9013,
17,Jamestown,US,78.12,overcast clouds,42.0970,-79.2353,
18,Padang,ID,76.66,overcast clouds,-0.9492,100.3543,
21,Kapaa,US,82.72,few clouds,22.0752,-159.3190,
30,Malartic,CA,80.42,broken clouds,48.1335,-78.1328,
38,Atuona,PF,79.30,few clouds,-9.8000,-139.0333,
41,Labuhan,ID,77.49,light rain,-6.8844,112.2051,
42,Albany,US,80.55,overcast clouds,42.6001,-73.9662,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('No hotel found... skipping!')
        
        

No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!


In [31]:
hotel_df.count()

City           225
Country        225
Max Temp       225
Description    225
Lat            225
Lng            225
Hotel Name     225
dtype: int64

In [39]:
# 7. Drop the rows where there is no Hotel Name. We should expect 15 rows to drop
hotel_df[hotel_df['Hotel Name'] == ''] #This code tells us there are 15 rows without hotel name
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df.count() # Now we have 15 less rows by dropping the ones that didn't find any hotel

City           210
Country        210
Max Temp       210
Description    210
Lat            210
Lng            210
Hotel Name     210
dtype: int64

In [41]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Korla,CN,77.56,overcast clouds,41.7597,86.1469,Gulisitan Hotel
8,Puerto Ayora,EC,76.96,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Havelock,US,88.05,clear sky,34.8791,-76.9013,Sherwood Motel
17,Jamestown,US,78.12,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
18,Padang,ID,76.66,overcast clouds,-0.9492,100.3543,Mercure Padang
21,Kapaa,US,82.72,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
30,Malartic,CA,80.42,broken clouds,48.1335,-78.1328,Camping Regional De Malartic
38,Atuona,PF,79.30,few clouds,-9.8000,-139.0333,Villa Enata
41,Labuhan,ID,77.49,light rain,-6.8844,112.2051,PT PPI Lamongan Tambak C
43,Pittsfield,US,79.81,clear sky,42.4501,-73.2454,"Holiday Inn & Suites Pittsfield-Berkshires, an..."


In [42]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))